# Importing required libraries

`sqlalchemy` is a "toolkit" that allows access to SQL databases through python. SQL tables can be queried and directly brought to python objects without the need of an intermediate export-import step (such as creating a csv from MySQL Workbench).

Converting data between incompatible type systems is called Object-relational mapping, so you might see `sqlalchemy` referred to as an ORM tool.

Here we'll see the basic workflow for connecting to a local database and transforming it to a `pandas` dataframe. Explore more features in the docs https://www.sqlalchemy.org/

MySQL has its own ORM tool for interacting with python, called `Connector/Python` (https://dev.mysql.com/doc/connector-python/en/connector-python-introduction.html). The advantage of `sqlalchemy` is that it can also work with other DBMS such as Oracle, Postgres, etc.

In [ ]:
#!pip3 install pymsql

#we need to install into our Conda DA_Env :

#conda install -c anaconda pymysql

#and

# conda install -c anaconda sqlalchemy

In [1]:
import pandas as pd
import getpass
import sqlalchemy as sa

# To connect and extract tables from SQL database.
Hint: the password is needed fot the DB user

In [4]:
driver = 'mysql+pymysql'
user = 'root'
password = getpass.getpass(prompt='Your password:'"")
ip = '127.0.0.1'  

Your password:········


In [5]:
connection_string = f'{driver}://{user}:{password}@{ip}'

In [6]:
db_connection = sa.create_engine(connection_string)

In [7]:
insp = sa.inspect(db_connection)
db_list = insp.get_schema_names()
print(db_list)

['bank', 'information_schema', 'lab_db', 'lab_db_python_sql', 'lab_group_by_db', 'mysql', 'Olist', 'performance_schema', 'publications', 'sakila', 'sys']


 Retreiving data into python object (without Pandas)

In [ ]:
result = db_connection.execute('SELECT * FROM bank.loan')
result
for row in result:
    print(row)

retrieve data from MYSQL into Pandas Data Frame

In [15]:
query="SELECT * FROM bank.loan Inner Join bank.Account on loan.account_id=account.account_id"
bank_loans = pd.read_sql_query(query, db_connection)
bank_loans.head()

,loan_id,account_id,date,amount,duration,payments,status,account_id,district_id,frequency,date
0,5314,1787,930705,96396,12,8033.0,B,1787,30,POPLATEK TYDNE,930322
1,5316,1801,930711,165960,36,4610.0,A,1801,46,POPLATEK MESICNE,930213
2,6863,9188,930728,127080,60,2118.0,A,9188,45,POPLATEK MESICNE,930208
3,5325,1843,930803,105804,36,2939.0,A,1843,12,POPLATEK MESICNE,930130
4,7240,11013,930906,274740,60,4579.0,A,11013,1,POPLATEK TYDNE,930214


Passing Parameter into a query

In [18]:
loan_id_threshold=5300
query = 'SELECT * FROM bank.loan where loan_id<'+str(loan_id_threshold)
filtered_loans = pd.read_sql_query(query, db_connection)
filtered_loans.head()

,loan_id,account_id,date,amount,duration,payments,status
0,4959,2,940105,80952,24,3373.0,A
1,5170,1071,940120,253200,60,4220.0,C
2,5285,1603,940206,78936,12,6578.0,A
3,5189,1166,940207,149040,48,3105.0,A
4,5130,813,940511,24312,12,2026.0,A


# Create a database in Mysql and save a table from Pandas into that database.

#### Create Database and commit

In [21]:
db_connection.execute("create database if not exists lab_db_python_sql;")

In [22]:
db_connection.execute("commit")

#### Inspect the database

In [23]:
insp = sa.inspect(db_connection)
db_list = insp.get_schema_names()
print(db_list)

['bank', 'information_schema', 'lab_db', 'lab_db_python_sql', 'lab_group_by_db', 'mysql', 'Olist', 'performance_schema', 'publications', 'sakila', 'sys']


#### Write Pandas Data Frame to Database

In [34]:
test_table = pd.DataFrame({"a" : [1,2,3], "b" : [4,5,6]})
test_table

,a,b
0,1,4
1,2,5
2,3,6


In [33]:
test_table.to_sql(name="clean_df", con=db_connection, schema="lab_db_python_sql")

#### Retreiving data from a View in a Sakilla database into pandas data frame

In [35]:
query="SELECT * FROM sakila.actor_info"
actor_info = pd.read_sql_query(query, db_connection)
actor_info

,actor_id,first_name,last_name,film_info
0,1,PENELOPE,GUINESS,Animation: ANACONDA CONFESSIONS; Children: LAN...
1,2,NICK,WAHLBERG,Action: BULL SHAWSHANK; Animation: FIGHT JAWBR...
2,3,ED,CHASE,"Action: CADDYSHACK JEDI, FORREST SONS; Classic..."
3,4,JENNIFER,DAVIS,Action: BAREFOOT MANCHURIAN; Animation: ANACON...
4,5,JOHNNY,LOLLOBRIGIDA,"Action: AMADEUS HOLY, GRAIL FRANKENSTEIN, RING..."
...,...,...,...,...
195,196,BELA,WALKEN,Action: FANTASY TROOPERS; Animation: LUKE MUMM...
196,197,REESE,WEST,Action: ANTITRUST TOMATOES; Animation: DOORS P...
197,198,MARY,KEITEL,"Action: FANTASY TROOPERS, FORREST SONS, HANDIC..."
198,199,JULIA,FAWCETT,"Action: BERETS AGENT; Animation: LUKE MUMMY, T..."


#### Calling Stored procedure from Sakilla Database

In [46]:
result = db_connection.execute('CALL sakila.film_in_stock(1, 1, @x);')
x = pd.read_sql_query('Select @x', db_connection)
x

,@x
0,4
